# TP3 - Market Basket Analysis
INF8111 - Fouille de données, Eté 2024
### Membres de l'équipe
    - Ettahri Mahacine - 2312965 - 1
    - Li Xuewei - 1879356 - 2
    - Sanjari Kaywan - 1989922 - 3


In [ ]:
bucket_name = 'bucket-tp3-inf8111'
ROOT = f'gs://{bucket_name}/'

## Date et directives de remise
Vous remettrez l'ensemble des fichiers nécessaires à la réalisation de TP au sein d'une archive compressée ZIP nommée TP3\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3

Cette archive devra, au minimum, inclure ce notebook, nommé TP3\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb

**Date limite: 23 juin à 23:55**.

#### Test de l'installation
À l'aide du code suivant, vous pouvez tester si Spark est correctement installé.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.sql("select 'spark' as hello ")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



## 4. MBA pour le dataset complet (20 points)


Comme vous l'avez probablement remarqué, même pour un ensemble de données moins volumineux, l'algorithme MBA est coûteux en calcul. Pour cette raison, cette fois, nous allons répéter le processus, mais en utilisant maintenant Google Cloud Platform (GCP) pour créer un grand cluster. Toutes les instructions pour créer un cluster avec spark et comment soumettre un travail seront expliquées dans le laboratoire. Dans tous les cas, vous devez lire les instructions données dans le ``Instruction_GCP.pdf``.

Cette fois, nous travaillerons avec le fichier ``OBSERVATIONS.csv``, qui contient plusieurs millions d'observations.

**PRODUCTION ATTENDUE**

Après avoir exécuté le MBA pour la plus grande collection d'observation, sélectionnez au hasard UN symptôme observé dans ``OBSERVATIONS`` et affichez les règles d'association (nom du symptôme et valeur d'association) de ce symptôme, c'est-à-dire lorsque le symptôme est seul. La sortie doit être formatée dans un tableau, où chaque ligne contenant les informations d'un symptôme associé.
Afin d'accélérer son exécution, vous pouvez créer un second notebook uniquement dédié à cette partie et l'insérer dans le rendu final.

- Affichez l'identifiant (SYMPTOM_ID) et le nom du symptôme (SYMPTOM_NAME) sélectionné au hasard.
- Signalez le temps d'exécution.

**Remarque importante : joignez des captures d'écran de votre sortie et de votre configuration de cluster.**

In [ ]:
%%time
"""
TODO: create a query to create and struct the transactions
"""
from itertools import combinations
from pyspark.sql.functions import collect_list, concat_ws, collect_set

def map_to_patterns(row):
    transaction = sorted(row.transaction.split(','))

    single = list(combinations(transaction, 1))
    pair = list(combinations(transaction, 2))
    trio = list(combinations(transaction, 3))

    pattern_list = single + pair + trio

    for p in pattern_list:
        yield (p, 1)

def format_tuples(pattern):
    return (str(pattern[0]), str(pattern[1]))

def reduce_patterns(v1, v2):
    return v1 + v2

def map_to_subpatterns(pattern):
    single = list(combinations(pattern[0], 1))
    pair = list(combinations(pattern[0], 2))
    trio = list(combinations(pattern[0], 3))

    combo_list = single + pair + trio
    combo_list_processed = [combo for combo in combo_list if len(combo) >= (len(pattern[0]) - 1)]

    for subpattern in combo_list_processed:
        missing_elements = set(pattern[0]) - set(subpattern)
        if missing_elements == set():
            missing_elements = None
            yield (subpattern, (None, pattern[1]))
        else:
            yield (subpattern, (*missing_elements, pattern[1]))


def map_to_assoc_rules(rule):
    # Unpack the input rule into subpattern and its associated rules
    subpattern, rules = rule

    # Extract the total count for the subpattern from the rules list
    # This is done by finding the count where the item is None
    total_count = next((count for item, count in rules if item is None), 0)

    # If total count is zero, return an empty list (no association rules possible)
    if total_count == 0:
        return []

    # Iterate over each item and its count in the rules list
    assoc_rules = [(item, count / total_count) for item, count in rules if item is not None]

    yield (subpattern, assoc_rules)

df_observations = spark.read.csv(ROOT + 'OBSERVATIONS.csv', header=True, sep=',', inferSchema=True)
df_symptoms = spark.read.csv(ROOT + 'SYMPTOMS.csv', header=True, sep=',', inferSchema=True)
df_OB_ext = df_observations.join(df_symptoms, ['SYMPTOM_ID'])
df_OB_ext = df_OB_ext.groupBy(df_OB_ext.VAERS_ID)\
                          .agg(concat_ws(',', collect_set('SYMPTOM_NAME')).alias('transaction'))\
                          .orderBy('VAERS_ID')


df_OB_ext.show(truncate = False)
df_OB_ext_rdd = df_OB_ext.rdd

+--------+-------------------------------------------------------------------------------------------------------------------------+
|VAERS_ID|transaction                                                                                                              |
+--------+-------------------------------------------------------------------------------------------------------------------------+
|25002   |Convulsion,Chills                                                                                                        |
|25010   |Bronchitis                                                                                                               |
|25011   |Vomiting,Convulsion,Pyrexia                                                                                              |
|25055   |Hepatic function abnormal,Hepatocellular damage,Nausea,Neoplasm malignant                                                |
|25056   |Hepatic cirrhosis,Malaise                                  

CPU times: user 38.5 ms, sys: 8.9 ms, total: 47.4 ms
Wall time: 21.1 s


In [ ]:
%%time
# Step 1 - Combination occurrences
df_OB_ext_rdd = df_OB_ext_rdd.flatMap(map_to_patterns)

CPU times: user 1.11 ms, sys: 455 µs, total: 1.56 ms
Wall time: 3.63 ms


In [ ]:
%%time
# Step 2 - ReduceByKey
df_OB_ext_rdd = df_OB_ext_rdd.reduceByKey(reduce_patterns)

CPU times: user 8.8 ms, sys: 1.95 ms, total: 10.8 ms
Wall time: 51.2 ms


In [ ]:
%%time
# Step 3 - Map to subpattern
df_OB_ext_rdd = df_OB_ext_rdd.flatMap(map_to_subpatterns)

CPU times: user 18 µs, sys: 7 µs, total: 25 µs
Wall time: 29.3 µs


In [ ]:
%%time
# Step 4 - Reduce Subpatterns
df_OB_ext_rdd = df_OB_ext_rdd.groupByKey().mapValues(list)

CPU times: user 6.45 ms, sys: 975 µs, total: 7.42 ms
Wall time: 16.5 ms


In [ ]:
%%time
# Step 5 - Map to Association Rules
df_OB_ext_rdd = df_OB_ext_rdd.flatMap(map_to_assoc_rules)

CPU times: user 18 µs, sys: 7 µs, total: 25 µs
Wall time: 29.1 µs


In [ ]:
%%time
df_OB_ext_rdd.map(format_tuples).toDF(['SYMPTOM (step 5)', 'ReduceSubpatterns']).sort(['SYMPTOM (step 5)']).show(20, truncate=False)

+--------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|SYMPTOM (step 5)                                                                                        |ReduceSubpatterns                                                                                                                                                                                                |
+--------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|("5'nucleotidase increased", 'Amylase normal')  

In [ ]:
%%time
import random
import ast
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, DoubleType

# Define a UDF to convert the string representation of a tuple to an array
def tuple_to_array(patterns):
    patterns = patterns.strip("()")
    array = [elem.strip(" '") for elem in patterns.split(",") if elem]
    return array

# Define a UDF to convert the string representation of association rules to an array of tuples
def assoc_rules_to_array(rules):
    return ast.literal_eval(rules)

# Register the UDFs
tuple_to_array_udf = F.udf(tuple_to_array, ArrayType(StringType()))
assoc_rules_to_array_udf = F.udf(assoc_rules_to_array, ArrayType(StructType([
    StructField("rule", StringType(), nullable=True),
    StructField("value", DoubleType(), nullable=True)
])))

random.seed(42)
# Collect a 1000 random rows from the RDD
random_1000_lines = df_OB_ext_rdd.sample(withReplacement=False, fraction=0.1, seed=42).take(1000)

# Parallelize these lines into a new RDD
random_1000_lines_rdd = spark.sparkContext.parallelize(random_1000_lines)

# Apply the UDFs and filter the DataFrame
assoc_rules_df_array = random_1000_lines_rdd.map(format_tuples).toDF(['patterns', 'association_rules'])\
    .withColumn("patterns", tuple_to_array_udf(F.col("patterns")))\
    .withColumn("association_rules", assoc_rules_to_array_udf(F.col("association_rules")))\
    .filter(F.size("patterns") == 1)

# Explode the patterns column
exploded_patterns_df = assoc_rules_df_array.select(F.explode("patterns").alias("pattern"))

# Collect the patterns into a list
patterns_list = exploded_patterns_df.select("pattern").rdd.flatMap(lambda x: x).collect()

# Choose a random pattern from the list)
random_symptom_name = random.choice(patterns_list)

# Filter the DataFrame for the chosen random pattern to get the corresponding association rules
association_rules_for_random_symptom = assoc_rules_df_array.filter(F.array_contains(F.col("patterns"), random_symptom_name))

# Split the association_rules column into separate rows
exploded_assoc_rules = association_rules_for_random_symptom.withColumn("association_rule", F.explode("association_rules"))
# Create a new DataFrame with 'symptom' and 'information' columns
association_table = exploded_assoc_rules.select(
     F.col("association_rule.rule").alias("symptom"),
     F.col("association_rule.value").alias("information")
)

# Get the Id of the chosen symptom from df_symptoms
symptom_row = df_symptoms.filter(df_symptoms.SYMPTOM_NAME == random_symptom_name.replace('"','')).select('SYMPTOM_ID','SYMPTOM_NAME').collect()[0]
random_symptom_id = symptom_row["SYMPTOM_ID"]

# Display the title with the randomly selected symptom ID and name
title = f"Association Rules for Symptom ID: {random_symptom_id}, Symptom Name: {random_symptom_name}"
print(title)

# Show the association rules table
association_table.show(truncate=False)

Association Rules for Symptom ID: 2545, Symptom Name: Tongue discolouration
+----------------------------+---------------------+
|symptom                     |information          |
+----------------------------+---------------------+
|Ureaplasma test positive    |0.002364066193853428 |
|Granulocyte percentage      |0.002364066193853428 |
|Tongue biting               |0.002364066193853428 |
|Secretion discharge         |0.0070921985815602835|
|Fluid retention             |0.002364066193853428 |
|Respiratory arrest          |0.002364066193853428 |
|Melaena                     |0.002364066193853428 |
|Ultrasound liver            |0.002364066193853428 |
|Musculoskeletal pain        |0.002364066193853428 |
|Coma                        |0.002364066193853428 |
|Chest discomfort            |0.04964539007092199  |
|Eating disorder             |0.01182033096926714  |
|Ageusia                     |0.04728132387706856  |
|Drug interaction            |0.002364066193853428 |
|Vascular insufficiency